In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from sklearn.utils import resample
from nltk.stem import WordNetLemmatizer
from PIL import Image, ImageFile
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from tensorflow.keras import Input, Model
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Layer, Embedding, LayerNormalization, MultiHeadAttention, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.layers import Layer, Dense, Dropout, Input, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Multiply, Dense, Layer, Dropout, LayerNormalization
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize


nltk.download('stopwords')
nltk.download('wordnet')

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
df = pd.read_csv('E:/MultiModal/memotion_dataset_7k/labels.csv')
df.head()

In [ ]:
df.drop(df.filter(regex='unnamed', axis=1), inplace=True, axis=1)
df.drop(columns=['text_ocr', 'humour', "motivational"], inplace=True)

In [ ]:
set_replace = {'positive': 0, 'neutral': 1, 'very_positive': 2, 'negative': 3, 'very_negative': 4}
df['overall_sentiment'] = df['overall_sentiment'].replace(set_replace)
cat_replace = {'not_offensive': 0, 'slight': 1, 'very_offensive': 2, 'hateful_offensive': 3}
df['offensive'] = df['offensive'].replace(cat_replace)
sar_replace = {'general': 0, 'twisted_meaning': 1, 'not_sarcastic': 2, 'very_twisted': 3}
df['sarcasm'] = df['sarcasm'].replace(sar_replace)

df.dropna(inplace=True)

rows_to_drop = ['image_120.jpg', 'image_4800.jpg', 'image_6782.jpg', 'image_6785.jpg', 
                'image_6787.jpg', 'image_6988.jpg', 'image_6989.jpg', 'image_6990.png',
                'image_6991.jpg', 'image_6992.jpg']
for image_name in rows_to_drop:
    df.drop(df[df['image_name'] == image_name].index, inplace=True)

width, height = 120, 120
image_folder = "E:/MultiModal/memotion_dataset_7k/images"

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['cleaned_text'] = df['text_corrected'].apply(clean_text)

In [ ]:
df = df[['cleaned_text', 'offensive', 'overall_sentiment', 'sarcasm', 'image_name']]
df.head()

In [ ]:
max_sentiment_size = df['overall_sentiment'].value_counts().max()
max_offensive_size = df['offensive'].value_counts().max()  
max_sarcasm_size = df['sarcasm'].value_counts().max()  
target_size = max(max_sentiment_size, max_offensive_size, max_sarcasm_size) 

def upsample_classes(df, target_column, target_size):
    classes = df[target_column].unique()
    resampled_list = []
    
    for class_value in classes:
        class_data = df[df[target_column] == class_value]
        resampled_class = resample(class_data,
                                   replace=True,  
                                   n_samples=target_size,
                                   random_state=42)
        resampled_list.append(resampled_class)
    
    return pd.concat(resampled_list)

df = upsample_classes(df, 'overall_sentiment', target_size)
df = upsample_classes(df, 'offensive', target_size)
df = upsample_classes(df, 'sarcasm', target_size)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(df['overall_sentiment'].value_counts())
print(df['offensive'].value_counts())
print(df['sarcasm'].value_counts())

In [ ]:
X = []
for image_name in tqdm(df['image_name']):
    path = os.path.join(image_folder, image_name)
    try:
        img = Image.open(path).convert('RGB').resize((width, height))
        img = np.array(img) / 255.0 
        X.append(img)
    except (OSError, IOError) as e:
        print(f"Error loading image {image_name}: {e}")

X = np.array(X)

print(f"Image data shape: {X.shape}")
y = df[['offensive', 'overall_sentiment', "sarcasm"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 60
text_data = df['cleaned_text'].values

In [ ]:
text_data = df['cleaned_text'].values

input_ids = []
attention_masks = []

for text in tqdm(text_data):
    encoded = tokenizer.encode_plus(
        text,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='np'
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])

input_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Masks shape: {attention_masks.shape}")

In [ ]:
input_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)

input_ids = np.squeeze(input_ids, axis=1)
attention_masks = np.squeeze(attention_masks, axis=1)
print(f"Input IDs shape after squeezing: {input_ids.shape}")
print(f"Attention Masks shape after squeezing: {attention_masks.shape}")

In [ ]:
X_train_text, X_test_text, train_masks, test_masks = train_test_split(
    input_ids, attention_masks, test_size=0.3, random_state=42
)

In [ ]:
y_train_sentiment = y_train['overall_sentiment']
y_train_offensive = y_train['offensive']
y_train_sarcasm = y_train['sarcasm']

y_test_sentiment = y_test['overall_sentiment']
y_test_offensive = y_test['offensive']
y_test_sarcasm = y_test['sarcasm']

In [ ]:
def lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    elif epoch >= 5 and epoch < 11:
        return lr * 0.5 
    else:
        return lr * 0.1 

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
class PositionalEncoding(Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.max_len = max_len
        self.d_model = d_model
        self.pos_encoding = self.positional_encoding(max_len, d_model)
    
    def get_angles(self, position, i, d_model):
        angles = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return position * angles
    
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
                                     np.arange(d_model)[np.newaxis, :],
                                     d_model)
        
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2]) 
        
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)
    
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "max_len": self.max_len,
            "d_model": self.d_model
        })
        return config


In [ ]:
class TransformerEncoder(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, inputs, training, mask=None):
        if mask is not None:
            mask = tf.expand_dims(mask, axis=1) 
            mask = tf.expand_dims(mask, axis=1) 

        attn_output = self.attention(inputs, inputs, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
def build_transformer_model(vocab_size, max_len, embed_dim, num_heads, ff_dim, num_layers):
    text_input = Input(shape=(max_len,), dtype=tf.int32, name='text_input')
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
    
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(text_input)
    positional_encoding_layer = PositionalEncoding(max_len, embed_dim)(embedding_layer)
    
    x = positional_encoding_layer
    for _ in range(num_layers):
        x = TransformerEncoder(embed_dim, num_heads, ff_dim)(x, mask=attention_mask)
    
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    
    return Model(inputs=[text_input, attention_mask], outputs=x)

In [ ]:
class PatchEmbedding(Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEmbedding, self).__init__()
        self.num_patches = num_patches
        self.projection_dim = projection_dim

    def build(self, input_shape):
        patch_size = input_shape[1] // int(np.sqrt(self.num_patches))
        self.flatten_patches = tf.keras.layers.Reshape(target_shape=(-1, patch_size * patch_size * input_shape[-1]))
        self.dense_projection = Dense(self.projection_dim)

    def call(self, images):
        patches = self.flatten_patches(images)
        projected_patches = self.dense_projection(patches)
        return projected_patches

In [ ]:
class ViT(Layer):
    def __init__(self, num_patches, projection_dim, num_heads, ff_dim, num_layers, residual=True, use_glu=True):
        super(ViT, self).__init__()
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_layers = num_layers
        self.residual = residual
        self.use_glu = use_glu

        # Patch and positional embeddings
        self.patch_embedding = PatchEmbedding(num_patches, projection_dim)
        self.position_embedding = PositionalEncoding(num_patches, projection_dim)

        # MultiHeadAttention and FeedForward layers for each Transformer layer
        self.attention_layers = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads + i,
                key_dim=projection_dim,
                attention_axes=(1,)  # Attention only across the patch dimension
            )
            for i in range(num_layers)
        ]

        # Initialize Dense layers for GLU outside the call method
        if self.use_glu:
            self.glu_dense_linear = [Dense(ff_dim) for _ in range(num_layers)]
            self.glu_dense_gate = [Dense(ff_dim, activation='sigmoid') for _ in range(num_layers)]
            self.glu_output_proj = [Dense(projection_dim) for _ in range(num_layers)]
        else:
            self.ffn_layers = [
                self.build_ffn_layer(projection_dim, ff_dim)
                for _ in range(num_layers)
            ]

        # Layer norms for pre-attention and pre-FFN normalization
        self.pre_layernorm_layers = [
            LayerNormalization(epsilon=1e-6)
            for _ in range(num_layers)
        ]
        self.post_layernorm_layers = [
            LayerNormalization(epsilon=1e-6)
            for _ in range(num_layers)
        ]

        # Adding multiple Dropout layers for regularization
        self.dropout_layers = [
            Dropout(0.1 + 0.05 * i)
            for i in range(num_layers)
        ]

    def build_ffn_layer(self, projection_dim, ff_dim):
        # Define the standard FFN layer outside the call method
        return tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(projection_dim)
        ])

    def gated_ffn(self, inputs, layer_idx):
        # Gated Linear Units (GLU) implementation
        linear_output = self.glu_dense_linear[layer_idx](inputs)
        gate = self.glu_dense_gate[layer_idx](inputs)
        gated_output = Multiply()([linear_output, gate])
        return self.glu_output_proj[layer_idx](gated_output)

    def call(self, inputs):
        # Initial embedding and position encoding
        x = self.patch_embedding(inputs)
        x = self.position_embedding(x)

        # Transformer layers with multi-head attention and FFN
        for i in range(self.num_layers):
            # Pre-Attention LayerNorm
            norm_x = self.pre_layernorm_layers[i](x)

            # Multi-Head Attention with residual connection
            attn_output = self.attention_layers[i](norm_x, norm_x)
            attn_output = self.dropout_layers[i](attn_output)
            if self.residual:
                x = x + attn_output  # Residual connection

            # Pre-FFN LayerNorm
            norm_x = self.post_layernorm_layers[i](x)

            # Feedforward network with possible GLU and residual
            if self.use_glu:
                ffn_output = self.gated_ffn(norm_x, i)
            else:
                ffn_output = self.ffn_layers[i](norm_x)

            ffn_output = self.dropout_layers[i](ffn_output)
            if self.residual:
                x = x + ffn_output  # Residual connection

        return x


In [ ]:
image_size = 120
patch_size = 10   
num_patches = (image_size // patch_size) ** 2
projection_dim = 128 

vocab_size = tokenizer.vocab_size + 1
embed_dim = 128
num_heads = 2
ff_dim = 512
num_layers = 2


In [ ]:
class LinearSelfAttentiveFusion(Layer):
    def __init__(self, projection_dim, **kwargs):
        super(LinearSelfAttentiveFusion, self).__init__(**kwargs)
        self.projection_dim = projection_dim
        self.W_q = Dense(projection_dim)  
        self.W_k = Dense(projection_dim)
        self.W_v = Dense(projection_dim) 
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, image_features, text_features):
        Q = self.W_q(image_features)
        K = self.W_k(text_features)
        V = self.W_v(text_features)

        attention_scores = tf.matmul(Q, K, transpose_b=True)
        attention_scores /= tf.sqrt(tf.cast(self.projection_dim, tf.float32))
        attention_weights = self.softmax(attention_scores)

        fusion_output = tf.matmul(attention_weights, V)

        return fusion_output

In [ ]:
image_size = 120
patch_size = 10   
num_patches = (image_size // patch_size) ** 2
projection_dim = 128
vocab_size = tokenizer.vocab_size + 1
embed_dim = 128
num_heads = 2
ff_dim = 512
num_layers = 2
max_len = 60 

In [ ]:
image_input = Input(shape=(image_size, image_size, 3), name='image_input')
x = ViT(num_patches, projection_dim, num_heads, ff_dim, num_layers)(image_input)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(128, activation='relu')(x)

text_input = Input(shape=(max_len,), dtype=tf.int32, name='text_input')
attention_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
text_features = build_transformer_model(vocab_size, max_len, embed_dim, num_heads, ff_dim, num_layers)([text_input, attention_mask])

combined = Concatenate()([x, text_features])

fusion_layer = LinearSelfAttentiveFusion(projection_dim)(x, text_features)

combined_fusion = Concatenate()([fusion_layer, combined])

sentiment_output = Dense(5, activation='softmax', name='sentiment_output')(combined_fusion)
offensive_output = Dense(4, activation='softmax', name='offensive_output')(combined_fusion)
sarcasm_output = Dense(4, activation='softmax', name='sarcasm_output')(combined_fusion)

model = Model(inputs=[image_input, text_input, attention_mask],
              outputs=[offensive_output, sentiment_output, sarcasm_output])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss={'sentiment_output': 'sparse_categorical_crossentropy',
                    'offensive_output': 'sparse_categorical_crossentropy',
                    "sarcasm_output": "sparse_categorical_crossentropy"},
              metrics=['accuracy'])

model.summary()

In [ ]:
# Fit the model
history = model.fit(
    [X_train, X_train_text, train_masks], 
    {'offensive_output': y_train_offensive, 'sentiment_output': y_train_sentiment, 'sarcasm_output': y_train_sarcasm},
    validation_data=([X_test, X_test_text, test_masks], 
                     {'offensive_output': y_test_offensive, 'sentiment_output': y_test_sentiment, 'sarcasm_output': y_test_sarcasm}),
    epochs=30,
    batch_size=16,
    callbacks=[early_stopping]
)

In [ ]:
history_dict = history.history
epochs = range(1, len(history_dict['loss']) + 1)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history_dict['loss'], label='Training Loss')
plt.plot(epochs, history_dict['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, history_dict['offensive_output_accuracy'],  label='Training Offensive Accuracy')
plt.plot(epochs, history_dict['val_offensive_output_accuracy'],  label='Validation Offensive Accuracy')
plt.plot(epochs, history_dict['sentiment_output_accuracy'], label='Training Sentiment Accuracy')
plt.plot(epochs, history_dict['val_sentiment_output_accuracy'],  label='Validation Sentiment Accuracy')
plt.plot(epochs, history_dict['sarcasm_output_accuracy'], label='Training Sarcasm Accuracy')
plt.plot(epochs, history_dict['val_sarcasm_output_accuracy'],  label='Validation Sarcasm Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Evaluate the model on the test data
eval_results = model.evaluate(
    [X_test, X_test_text, test_masks], 
    {
        'offensive_output': y_test_offensive, 
        'sentiment_output': y_test_sentiment, 
        'sarcasm_output': y_test_sarcasm
    }
)

# Extracting loss and accuracy metrics from evaluation results
test_loss = eval_results[0]  # Total loss
offensive_output_loss = eval_results[1]  # Loss for offensive task
sentiment_output_loss = eval_results[2]  # Loss for sentiment task
sarcasm_output_loss = eval_results[3]    # Loss for sarcasm task

offensive_output_accuracy = eval_results[4]  # Accuracy for offensive task
sentiment_output_accuracy = eval_results[5]  # Accuracy for sentiment task
sarcasm_output_accuracy = eval_results[6]    # Accuracy for sarcasm task

# Printing results
print(f"Test Loss: {test_loss}")
print(f"Offensive Output Loss: {offensive_output_loss}")
print(f"Sentiment Output Loss: {sentiment_output_loss}")
print(f"Sarcasm Output Loss: {sarcasm_output_loss}")

print(f"Offensive Output Accuracy: {offensive_output_accuracy}")
print(f"Sentiment Output Accuracy: {sentiment_output_accuracy}")
print(f"Sarcasm Output Accuracy: {sarcasm_output_accuracy}")

In [ ]:
# Get model predictions
y_pred = model.predict([X_test, X_test_text, test_masks])

# Since the model has three outputs, get the predictions for each output
y_pred_offensive = np.argmax(y_pred[0], axis=1)
y_pred_sentiment = np.argmax(y_pred[1], axis=1)
y_pred_sarcasm = np.argmax(y_pred[2], axis=1)

# Print classification reports for each output
print("\n--- Offensive Output Classification Report ---")
print(classification_report(y_test_offensive, y_pred_offensive, target_names=["Class 0", "Class 1", "Class 2", "Class 3"]))

print("\n--- Sentiment Output Classification Report ---")
print(classification_report(y_test_sentiment, y_pred_sentiment, target_names=["Class 0", "Class 1", "Class 2", "Class 3", "Class 4"]))

print("\n--- Sarcasm Output Classification Report ---")
print(classification_report(y_test_sarcasm, y_pred_sarcasm, target_names=["Class 0", "Class 1", "Class 2", "Class 3"]))

# Macro metrics
macro_precision_offensive = precision_score(y_test_offensive, y_pred_offensive, average='macro')
macro_recall_offensive = recall_score(y_test_offensive, y_pred_offensive, average='macro')
macro_f1_offensive = f1_score(y_test_offensive, y_pred_offensive, average='macro')

macro_precision_sentiment = precision_score(y_test_sentiment, y_pred_sentiment, average='macro')
macro_recall_sentiment = recall_score(y_test_sentiment, y_pred_sentiment, average='macro')
macro_f1_sentiment = f1_score(y_test_sentiment, y_pred_sentiment, average='macro')

macro_precision_sarcasm = precision_score(y_test_sarcasm, y_pred_sarcasm, average='macro')
macro_recall_sarcasm = recall_score(y_test_sarcasm, y_pred_sarcasm, average='macro')
macro_f1_sarcasm = f1_score(y_test_sarcasm, y_pred_sarcasm, average='macro')

# Micro metrics
micro_precision_offensive = precision_score(y_test_offensive, y_pred_offensive, average='micro')
micro_recall_offensive = recall_score(y_test_offensive, y_pred_offensive, average='micro')
micro_f1_offensive = f1_score(y_test_offensive, y_pred_offensive, average='micro')

micro_precision_sentiment = precision_score(y_test_sentiment, y_pred_sentiment, average='micro')
micro_recall_sentiment = recall_score(y_test_sentiment, y_pred_sentiment, average='micro')
micro_f1_sentiment = f1_score(y_test_sentiment, y_pred_sentiment, average='micro')

micro_precision_sarcasm = precision_score(y_test_sarcasm, y_pred_sarcasm, average='micro')
micro_recall_sarcasm = recall_score(y_test_sarcasm, y_pred_sarcasm, average='micro')
micro_f1_sarcasm = f1_score(y_test_sarcasm, y_pred_sarcasm, average='micro')

# Print macro metrics
print("\n--- Offensive Output Macro Metrics ---")
print(f"Macro Precision: {macro_precision_offensive}")
print(f"Macro Recall: {macro_recall_offensive}")
print(f"Macro F1: {macro_f1_offensive}")

print("\n--- Sentiment Output Macro Metrics ---")
print(f"Macro Precision: {macro_precision_sentiment}")
print(f"Macro Recall: {macro_recall_sentiment}")
print(f"Macro F1: {macro_f1_sentiment}")

print("\n--- Sarcasm Output Macro Metrics ---")
print(f"Macro Precision: {macro_precision_sarcasm}")
print(f"Macro Recall: {macro_recall_sarcasm}")
print(f"Macro F1: {macro_f1_sarcasm}")

# Print micro metrics
print("\n--- Offensive Output Micro Metrics ---")
print(f"Micro Precision: {micro_precision_offensive}")
print(f"Micro Recall: {micro_recall_offensive}")
print(f"Micro F1: {micro_f1_offensive}")

print("\n--- Sentiment Output Micro Metrics ---")
print(f"Micro Precision: {micro_precision_sentiment}")
print(f"Micro Recall: {micro_recall_sentiment}")
print(f"Micro F1: {micro_f1_sentiment}")

print("\n--- Sarcasm Output Micro Metrics ---")
print(f"Micro Precision: {micro_precision_sarcasm}")
print(f"Micro Recall: {micro_recall_sarcasm}")
print(f"Micro F1: {micro_f1_sarcasm}")


In [ ]:
y_pred = model.predict([X_test, X_test_text, test_masks])

y_pred_offensive = y_pred[0]
y_pred_sentiment = y_pred[1]
y_pred_sarcasm = y_pred[2]

y_pred_offensive_classes = np.argmax(y_pred_offensive, axis=-1)
y_pred_sentiment_classes = np.argmax(y_pred_sentiment, axis=-1)
y_pred_sarcasm_classes = np.argmax(y_pred_sarcasm, axis=-1)

In [ ]:
cm_offensive = confusion_matrix(y_test_offensive, y_pred_offensive_classes)
cm_sentiment = confusion_matrix(y_test_sentiment, y_pred_sentiment_classes)
cm_sarcasm = confusion_matrix(y_test_sarcasm, y_pred_sarcasm_classes)

plt.figure(figsize=(3, 3))

sns.heatmap(cm_offensive, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['not_offensive', 'slight', 'very_offensive', 'hateful_offensive'], 
            yticklabels=['not_offensive', 'slight', 'very_offensive', 'hateful_offensive'])
plt.title('Confusion Matrix - Offensive')
plt.show()
plt.figure(figsize=(3, 3))

sns.heatmap(cm_sentiment, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['positive', 'neutral', 'very_positive', 'negative', 'very_negative'], 
            yticklabels=['positive', 'neutral', 'very_positive', 'negative', 'very_negative'])
plt.title('Confusion Matrix - Sentiment')
sar_replace = {'general': 0, 'twisted_meaning': 1, 'not_sarcastic': 2, 'very_twisted': 3}

plt.show()
plt.figure(figsize=(3, 3))

sns.heatmap(cm_sarcasm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['general', 'twisted_meaning', 'not_sarcastic', 'very_twisted'], 
            yticklabels=['general', 'twisted_meaning', 'not_sarcastic', 'very_twisted'])
plt.title('Sarcasm Matrix - Scarcasm')
plt.show()


In [ ]:
y_test_offensive_bin = label_binarize(y_test_offensive, classes=[0, 1, 2, 3])
y_test_sentiment_bin = label_binarize(y_test_sentiment, classes=[0, 1, 2, 3, 4])
y_test_sarcasm_bin = label_binarize(y_test_sarcasm, classes=[0, 1, 2, 3])

y_pred_offensive_probs = model.predict([X_test, X_test_text, test_masks])[0] 
y_pred_sentiment_probs = model.predict([X_test, X_test_text, test_masks])[1] 
y_pred_sarcasm_probs = model.predict([X_test, X_test_text, test_masks])[2] 

offensive_classes = ['not_offensive', 'slight', 'very_offensive', 'hateful_offensive']
sentiment_classes = ['positive', 'neutral', 'very_positive', 'negative', 'very_negative']
sarcasm_classes = ['general', 'twisted_meaning', 'not_sarcastic', 'very_twisted']


In [ ]:
plt.figure(figsize=(4, 4))

for i, class_name in enumerate(offensive_classes):
    fpr, tpr, _ = roc_curve(y_test_offensive_bin[:, i], y_pred_offensive_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{class_name} (area = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.title('Class-Specific ROC Curve - Offensive')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')

plt.figure(figsize=(4, 4))
for i, class_name in enumerate(sentiment_classes):
    fpr, tpr, _ = roc_curve(y_test_sentiment_bin[:, i], y_pred_sentiment_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{class_name} (area = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.title('Class-Specific ROC Curve - Sentiment')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()


plt.figure(figsize=(4, 4))
for i, class_name in enumerate(sarcasm_classes):
    fpr, tpr, _ = roc_curve(y_test_sarcasm_bin[:, i], y_pred_sarcasm_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{class_name} (area = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.title('Class-Specific ROC Curve - Sarcasm')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()